<a href="https://colab.research.google.com/github/OrtegaJulio7/SIS420/blob/main/Final2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reinforcement learning (Aprendizaje por refuerzo)

El aprendizaje por refuerzo (RL) es una técnica de machine learning (ML) que entrena al software para que tome decisiones a fin de lograr los mejores resultados. Imita el proceso de aprendizaje por ensayo y error que los humanos utilizan para lograr sus objetivos.
<HR>
 <h3>
  NOMBRE:
  * ORTEGA ALBARADO JULIO CESAR<br>
  
  CARRERA: INGENIERIA DE SISTEMAS <BR>
  
  * [Enlace al git hub](https://github.com/OrtegaJulio7/SIS420/blob/main/Final.ipynb)

 <h3>

# Descripcion del proyecto:Juego de Navegación con Q-learning en un Entorno Dinámico
El proyecto tiene como objetivo crear un juego de navegación en el que un agente(prisionero) aprende a moverse en un entorno dinámico utilizando el algoritmo de Q-learning, una técnica de aprendizaje por refuerzo. El entorno está diseñado para ser desafiante, con obstáculos, trampas y un guardia que persigue al agente, haciendo que el agente tenga que tomar decisiones inteligentes para lograr escapar o alcanzar una meta (la salida) sin ser atrapado.

## Entorno del Juego
* El entorno es una cuadrícula de 5x5 (puede ser ampliado según se desee).
* El agente puede moverse en 4 direcciones: arriba, abajo, izquierda y derecha.
* Las posiciones de la trampa, el guardia y la salida se generan aleatoriamente al inicio de cada episodio.
* El guardia se mueve aleatoriamente dentro del entorno, representando un peligro para el agente.
* El agente obtiene una recompensa positiva por alcanzar la salida y penalizaciones por caer en la trampa o ser atrapado por el guardia.

# 1. Crear el entorno dinámico

In [1]:
import random
import numpy as np

# Definir el tamaño del entorno (por ejemplo, una cuadrícula 5x5)
tamaño = (5, 5)

# Definir posiciones especiales
def generar_posiciones_dinamicas():
    """
    Genera posiciones aleatorias para los obstáculos, el guardia y la salida en el entorno.
    """
    trampa = (random.randint(0, tamaño[0]-1), random.randint(0, tamaño[1]-1))
    guardia = (random.randint(0, tamaño[0]-1), random.randint(0, tamaño[1]-1))

    while trampa == guardia:  # Asegurarse de que no haya superposición entre trampa y guardia
        guardia = (random.randint(0, tamaño[0]-1), random.randint(0, tamaño[1]-1))

    salida = (random.randint(0, tamaño[0]-1), random.randint(0, tamaño[1]-1))

    return trampa, guardia, salida

def mover_guardia(guardia):
    """
    Mueve el guardia aleatoriamente en el entorno.
    """
    mov = random.choice([(0, 1), (0, -1), (1, 0), (-1, 0)])  # Movimiento aleatorio
    nueva_pos = (guardia[0] + mov[0], guardia[1] + mov[1])
    nueva_pos = (max(0, min(nueva_pos[0], tamaño[0]-1)), max(0, min(nueva_pos[1], tamaño[1]-1)))  # Asegurarse que no salga del entorno
    return nueva_pos

def obtener_recompensa(estado, guardia, trampa, salida):
    """
    Calcula la recompensa en función de la ubicación del agente.
    """
    if estado == salida:
        return 100  # Recompensa por llegar a la salida
    elif estado == trampa:
        return -50  # Penalización por caer en la trampa
    elif estado == guardia:
        return -20  # Penalización por encontrarse con el guardia
    else:
        return -1  # Penalización por moverse a cualquier otro estado (costo de movimiento)


# 2. Implementación de Q-learning con entorno dinámico
Implementamos el algoritmo de Q-learning, donde el agente aprenderá a tomar decisiones basadas en la tabla Q. El entorno es dinámico y cambiará a medida que el agente avanza.

In [2]:
# Inicialización de la tabla Q (por ejemplo, para una cuadrícula 5x5)
Q = np.zeros((tamaño[0], tamaño[1], 4))  # 4 acciones: arriba, abajo, izquierda, derecha

# Parámetros de Q-learning
alpha = 0.1  # Tasa de aprendizaje
gamma = 0.9  # Factor de descuento
epsilon = 0.3  # Tasa de exploración
n_episodios = 500  # Número de episodios de entrenamiento

# Definir las acciones (0: Arriba, 1: Abajo, 2: Izquierda, 3: Derecha)
acciones = [(0, 1), (0, -1), (-1, 0), (1, 0)]  # Movimiento en (fila, columna)

# Función para seleccionar acción (exploración o explotación) E-greedy
def seleccionar_accion(estado):
    if random.uniform(0, 1) < epsilon:  # Exploración
        return random.choice([0, 1, 2, 3])  # Seleccionar una acción aleatoria
    else:  # Explotación
        return np.argmax(Q[estado[0], estado[1]])  # Elegir la acción con mayor valor Q

# Función para actualizar la tabla Q
def actualizar_Q(estado, accion, recompensa, nuevo_estado):
    # Q-Update Rule: Q(s, a) <- Q(s, a) + alpha * (recompensa + gamma * max(Q(s', a)) - Q(s, a))
    max_Q = np.max(Q[nuevo_estado[0], nuevo_estado[1]])  # Mejor valor Q del siguiente estado
    Q[estado[0], estado[1], accion] += alpha * (recompensa + gamma * max_Q - Q[estado[0], estado[1], accion])

# Función para entrenar al agente en el entorno dinámico
def entrenar_agente():
    for episodio in range(n_episodios):
        # Inicializar el entorno dinámicamente en cada episodio
        trampa, guardia, salida = generar_posiciones_dinamicas()
        estado = (random.randint(0, tamaño[0]-1), random.randint(0, tamaño[1]-1))  # Estado inicial aleatorio

        total_recompensa = 0
        pasos = 0

        while estado != salida and pasos < 50:  # Limitar el número de pasos para evitar bucles infinitos
            accion = seleccionar_accion(estado)  # Elegir una acción
            nuevo_estado = (estado[0] + acciones[accion][0], estado[1] + acciones[accion][1])

            # Asegurarse de que el nuevo estado esté dentro de los límites del entorno
            nuevo_estado = (max(0, min(nuevo_estado[0], tamaño[0]-1)), max(0, min(nuevo_estado[1], tamaño[1]-1)))

            # Obtener recompensa dinámica
            recompensa = obtener_recompensa(nuevo_estado, guardia, trampa, salida)

            # Actualizar la tabla Q
            actualizar_Q(estado, accion, recompensa, nuevo_estado)

            # Mover al guardia y actualizar la recompensa
            guardia = mover_guardia(guardia)

            # Pasar al siguiente estado
            estado = nuevo_estado
            total_recompensa += recompensa
            pasos += 1

        print(f"Episodio {episodio + 1}/{n_episodios} - Recompensa total: {total_recompensa}")

# Entrenamiento del agente
entrenar_agente()


Episodio 1/500 - Recompensa total: -167
Episodio 2/500 - Recompensa total: -107
Episodio 3/500 - Recompensa total: -126
Episodio 4/500 - Recompensa total: 75
Episodio 5/500 - Recompensa total: -156
Episodio 6/500 - Recompensa total: -69
Episodio 7/500 - Recompensa total: -118
Episodio 8/500 - Recompensa total: -88
Episodio 9/500 - Recompensa total: -107
Episodio 10/500 - Recompensa total: 92
Episodio 11/500 - Recompensa total: -50
Episodio 12/500 - Recompensa total: -69
Episodio 13/500 - Recompensa total: -69
Episodio 14/500 - Recompensa total: -167
Episodio 15/500 - Recompensa total: -197
Episodio 16/500 - Recompensa total: -99
Episodio 17/500 - Recompensa total: 99
Episodio 18/500 - Recompensa total: -25
Episodio 19/500 - Recompensa total: 79
Episodio 20/500 - Recompensa total: -107
Episodio 21/500 - Recompensa total: -85
Episodio 22/500 - Recompensa total: 0
Episodio 23/500 - Recompensa total: -167
Episodio 24/500 - Recompensa total: 69
Episodio 25/500 - Recompensa total: 3
Episodio

In [3]:
# Imprimir la tabla Q después del entrenamiento
def imprimir_Q():
    for i in range(tamaño[0]):
        for j in range(tamaño[1]):
            print(f"Estado ({i}, {j}): {Q[i, j]}")

imprimir_Q()


Estado (0, 0): [  2.7535697   -7.65278755  -8.32552483 -11.29148882]
Estado (0, 1): [ -6.04137703 -10.85239744  -6.59955828  -1.92874338]
Estado (0, 2): [-12.84848927  -6.69899875  -4.98062952  -1.23067979]
Estado (0, 3): [-10.90962899  -7.66201319  -7.01429063  -5.35790013]
Estado (0, 4): [-5.15292084 -5.94409333 -7.10552633 -4.78815263]
Estado (1, 0): [-6.51943109 -6.79858434 -0.35150916 -8.57406476]
Estado (1, 1): [-0.59827264 -3.25817    -5.7631265  -1.13350376]
Estado (1, 2): [-3.96990265 -2.1682589  -2.19686864 -1.03185322]
Estado (1, 3): [-3.62786015 -0.91753539 -6.53382871 -5.60060277]
Estado (1, 4): [-4.39845571 -4.461936   -5.97860264 -8.32230636]
Estado (2, 0): [-6.12989454 -4.58847192  3.80413024  1.59460844]
Estado (2, 1): [-3.67492995  0.62745213 -3.10499493 -4.98487745]
Estado (2, 2): [-1.54024324 -3.13608759 -2.50083301 -3.27730115]
Estado (2, 3): [-11.22626373  -1.39098606  -3.31068098  -4.07659252]
Estado (2, 4): [-13.51429286  -2.99018783  -0.49601529  -5.51692476]
E

# Prueba de explotación para 100 episodios

In [5]:
import random
import numpy as np

# Tamaño del entorno y la tabla Q entrenada
tamaño = (5, 5)
acciones = [(0, 1), (0, -1), (-1, 0), (1, 0)]  # Arriba, Abajo, Izquierda, Derecha

# Definir posiciones dinámicas: trampa, guardia, salida
def generar_posiciones_dinamicas():
    trampa = (random.randint(0, tamaño[0]-1), random.randint(0, tamaño[1]-1))
    guardia = (random.randint(0, tamaño[0]-1), random.randint(0, tamaño[1]-1))
    while trampa == guardia:  # Asegurarse de que no haya superposición
        guardia = (random.randint(0, tamaño[0]-1), random.randint(0, tamaño[1]-1))
    salida = (random.randint(0, tamaño[0]-1), random.randint(0, tamaño[1]-1))
    return trampa, guardia, salida

# Función para obtener recompensa dinámica
def obtener_recompensa(estado, guardia, trampa, salida):
    if estado == salida:
        return 100  # Recompensa por llegar a la salida
    elif estado == trampa:
        return -50  # Penalización por caer en la trampa
    elif estado == guardia:
        return -20  # Penalización por encontrarse con el guardia
    else:
        return -1  # Penalización por moverse a cualquier otro estado

# Función para mover al guardia
def mover_guardia(guardia):
    mov = random.choice([(0, 1), (0, -1), (1, 0), (-1, 0)])  # Movimiento aleatorio
    nueva_pos = (guardia[0] + mov[0], guardia[1] + mov[1])
    nueva_pos = (max(0, min(nueva_pos[0], tamaño[0]-1)), max(0, min(nueva_pos[1], tamaño[1]-1)))
    return nueva_pos

# Función para probar el agente (sin exploración)
def probar_agente_explotacion(Q, n_episodios=100):
    for episodio in range(n_episodios):
        trampa, guardia, salida = generar_posiciones_dinamicas()
        estado = (random.randint(0, tamaño[0]-1), random.randint(0, tamaño[1]-1))  # Estado inicial aleatorio

        atrapado = False
        pasos = 0

        while estado != salida and pasos < 50:  # Limitar pasos para evitar bucles infinitos
            # Seleccionar acción con la mayor Q (explotación)
            accion = np.argmax(Q[estado[0], estado[1]])
            nuevo_estado = (estado[0] + acciones[accion][0], estado[1] + acciones[accion][1])

            # Asegurarse de que el nuevo estado esté dentro de los límites del entorno
            nuevo_estado = (max(0, min(nuevo_estado[0], tamaño[0]-1)), max(0, min(nuevo_estado[1], tamaño[1]-1)))

            # Obtener la recompensa dinámica
            recompensa = obtener_recompensa(nuevo_estado, guardia, trampa, salida)

            # Mover al guardia
            guardia = mover_guardia(guardia)

            # Verificar si el agente ha sido atrapado
            if nuevo_estado == guardia:
                atrapado = True
                break  # El agente es atrapado, termina el episodio

            # Actualizar el estado
            estado = nuevo_estado
            pasos += 1

        # Mostrar el resultado del episodio
        if atrapado:
            print(f"Episodio {episodio + 1}/{n_episodios} - El agente fue atrapado")
        else:
            print(f"Episodio {episodio + 1}/{n_episodios} - El agente llegó a la salida")

# Probar el agente con explotación
probar_agente_explotacion(Q, n_episodios=100)


Episodio 1/100 - El agente llegó a la salida
Episodio 2/100 - El agente fue atrapado
Episodio 3/100 - El agente fue atrapado
Episodio 4/100 - El agente llegó a la salida
Episodio 5/100 - El agente llegó a la salida
Episodio 6/100 - El agente fue atrapado
Episodio 7/100 - El agente llegó a la salida
Episodio 8/100 - El agente fue atrapado
Episodio 9/100 - El agente fue atrapado
Episodio 10/100 - El agente fue atrapado
Episodio 11/100 - El agente llegó a la salida
Episodio 12/100 - El agente llegó a la salida
Episodio 13/100 - El agente llegó a la salida
Episodio 14/100 - El agente llegó a la salida
Episodio 15/100 - El agente fue atrapado
Episodio 16/100 - El agente llegó a la salida
Episodio 17/100 - El agente fue atrapado
Episodio 18/100 - El agente fue atrapado
Episodio 19/100 - El agente fue atrapado
Episodio 20/100 - El agente fue atrapado
Episodio 21/100 - El agente fue atrapado
Episodio 22/100 - El agente llegó a la salida
Episodio 23/100 - El agente llegó a la salida
Episodio 24